# 🔊 Elevating Trailers with Automated Narration

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Elevenlabs_Voiceover_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction 

Narration is the heartbeat of trailers, injecting excitement and intrigue into every frame. With [VideoDB](https://videodb.io), [OpenAI](https://openai.com), and [ElevenLabs](https://elevenlabs.io) at your fingertips, adding narration to trailers becomes a thrilling adventure. This tutorial will guide you through the exhilarating process of seamlessly integrating narration into trailers using these powerful tools.

## Setup

---

### 📦  Installing packages 

In [ ]:
%pip install openai
%pip install videodb

### 🔑 API keys
Before proceeding, ensure access to [VideoDB](https://videodb.io), [OpenAI](https://openai.com), and [ElevenLabs](https://elevenlabs.io) API key. If not, sign up for API access on the respective platforms.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [13]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["ELEVEN_LABS_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

### 🎙️  ElevenLab's Voice ID  
You will also need ElevenLab's VoiceID of a Voice that you want to use.

For this demo, we will be using [Sam Elliot's Voice](https://elevenlabs.io/app/voice-lab/share/bcf1f77ee30ba698ec2808b47188b4189a69c862c073d704fde7034608eeaec5/nRjJQMMnjuyMXVyDw9TC). ElevenLabs has a large variety of voices to choose from (browse them [here](https://elevenlabs.io/voice-library)). Once finalized, copy the Voice ID from ElevenLabs and link it here.

In [15]:
voiceover_artist_id = "VOICEOVER_ARTIST_ID"

## Tutorial Walkthrough

---


### 📋 Step 1: Connect to VideoDB
Gear up by establishing a connection to VideoDB using your API key to manage and make the most of you video library with unparalleled efficiency.

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()

### 🎬 Step 2: Upload the Trailer
Upload the trailer video to VideoDB for further processing. This creates the base video asset that we shall use later in this tutorial.

In [5]:
video = conn.upload(url='https://www.youtube.com/watch?v=WQmGwmc-XUY')

### 🔍 Step 3: Analyze Scenes and Generate Scene Descriptions

Start by analyzing the scenes within your Video using VideoDB's scene indexing capabilities. This will provide context for generating the script prompt.

In [6]:
video.index_scenes()

Let's view the description of first scene from the video

In [8]:
scenes = video.get_scenes()
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

0 - 0.7090416666666666
The image captures a fiery blaze, a dynamic dance of flames in vivid shades of orange, gold, and red. Light flickers intensely, radiance expanding, contracting with the fire's rhythm. No specific source is visible; the fire dominates entirely, filling the frame with energetic movement. The luminosity suggests a fierce heat, powerful enough to demand respect and caution. Each tongue of flame is seemingly alive, almost writhing against a darker, indistinct background. This could be a natural fire or a controlled blaze—there’s no context to indicate its origin. Amidst the searing heat, the flames create a mesmeric, albeit destructive, spectacle.


### 🔊 Step 4: Generate Narration Script with LLM
Here, we use OpenAI’s GPT to build context around the scene descriptions above, and generate a fitting narration script for the visuals.

In [9]:
# Generate narration script with ChatGPT
import openai

client = openai.OpenAI()

script_prompt = "Craft a dynamic narration script for this trailer, incorporating scene descriptions to enhance storytelling. Ensure that the narration aligns seamlessly with the timestamps provided in the scene index. Do not include any annotations in the output script"

full_prompt = script_prompt + "\n\n"
for scene in scenes:
  full_prompt += f"- {scene}\n"

openai_res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": full_prompt}],
)
voiceover_script = openai_res.choices[0].message.content

# If you have ElevenLab's paid plan remove the :2500 limit on voiceover script.
voiceover_script = voiceover_script[:2500]

You can refine the narration script prompt to ensure synchronization with timestamps in the scene index, optimizing the storytelling experience.

### 🎙️ Step 5: Generate Narration Audio with elevenlabs.io
Prepare to be dazzled as you synthesize the narration audio using elevenlabs.io API, tailored to match the trailer's pulse-pounding rhythm.

Note: for this step, you will need a specific voice ID that fits perfectly with the vibe of your trailer. In our example, we have used this voice that resembles the vocal quality and style that of Sam Elliott. You can find a voice suitable for your trailer in the ElevenLabs Voice Library.

In [ ]:
import requests


# Call ElevenLabs API to generate voiceover
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voiceover_artist_id}"
headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "model_id": "eleven_monolingual_v1",
    "text": voiceover_script,
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}
elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)

# Save the audio file
audio_file = "audio.mp3"
CHUNK_SIZE = 1024
with open(audio_file, 'wb') as f:
    for chunk in elevenlabs_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

### 🎬 Step 6: Add Voiceover to Video with VideoDB

Upload the audio file (voiceover) to VideoDB

In [ ]:
audio = conn.upload(file_path=audio_file)

### 🎥 Step 7: Add Narration to Trailer with VideoDB
Combine the narration audio with the trailer using VideoDB's timeline feature.

In [ ]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

timeline = Timeline(conn)

video_asset = VideoAsset(asset_id=video.id, start=0)

audio_asset1 = AudioAsset(asset_id=audio.id, start=5, end=25.5, disable_other_tracks=False)
audio_asset2 = AudioAsset(asset_id=audio.id, start=35, end=49, disable_other_tracks=False)

# add asset overlay
timeline.add_inline(asset=video_asset)
timeline.add_overlay(start=4, asset=audio_asset1)
timeline.add_overlay(start=35, asset=audio_asset2)


### 🪄 Step 8: Review and Share
Preview the trailer with the integrated narration to ensure it aligns with your vision. Once satisfied, share the trailer with others to experience the enhanced storytelling.


In [15]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/fea6943a-0ac6-4a13-9309-4df35e41f6f7.m3u8'

## 🎬 Bonus
-----

Add Movie Poster at the End

To incorporate adding a movie poster as a bonus section at the end of the trailer using VideoDB, you can follow these steps (you’ll find the code snippet for the same below):

1. We first import the necessary module for handling Image Assets. 
2. Then, we upload the movie poster image to VideoDB. 
3. Next, we create an Image Asset with the uploaded poster image, specifying its dimensions and duration. 
4. Finally, we add the movie poster as an overlay at the end of the video timeline.

In [ ]:
from videodb.asset import ImageAsset
from videodb import MediaType, play_stream

image = conn.upload(url="https://raw.githubusercontent.com/video-db/videodb-cookbook/dev/images/chase_trailer.png", media_type=MediaType.image)

image_asset = ImageAsset(
    asset_id=image.id,
    width=1392,
    height=783,
    x=0,
    y=0,
    duration=13
)

timeline.add_overlay(start=67.5, asset=image_asset)

stream_url = timeline.generate_stream()
play_stream(stream_url)

### 🎉 Conclusion
____
Congratulations! You're about to dive into the exciting world of adding narration to trailers with VideoDB, OpenAI, and elevenlabs.io. Get ready to unleash your creativity and captivate audiences with thrilling trailers. Try out various narration styles and prompts to personalize your trailers and keep viewers hooked. Get excited about enhancing your trailers with AI-powered narration using VideoDB!

For more such explorations, refer to the [VideoDB docs](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.

We're excited to see your creations, so we welcome you to share your creations via [Discord](https://discord.com/invite/py9P639jGz), [LinkedIn](https://www.linkedin.com/company/videodb) or [Twitter](https://twitter.com/videodb_io).


In the meanwhile, check out the other exciting outputs generated using this tutorial and share yours in our [community](https://discord.com/invite/py9P639jGz) too! We’d love to feature you here ⚡️